https://medium.com/@ksusorokina/image-classification-with-convolutional-neural-networks-496815db12a8

In [3]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import imageio as imageio
import os, random
import matplotlib.pyplot as plt

datagen = ImageDataGenerator(
        width_shift_range=0.2,
        height_shift_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

#no rotation, skewing or shearing

img = load_img('images/z0.jpg')  
x = img_to_array(img)
print(x.shape)
x = x.reshape((1,) + x.shape)
x.shape
print(x.shape)

#image loaded

i = 0
for batch in datagen.flow(x, batch_size=1,
                         save_to_dir='preview', save_prefix='z', save_format='jpeg'):
    i += 1
    if i > 3: #extra generated image count
        break
        
#augmented data generated

random_file = "preview/"+random.choice(os.listdir("preview"))
print(random_file)
image = imageio.imread(random_file)
plt.imshow(image)
plt.show()
print(image.shape)

(183, 275, 3)
(1, 183, 275, 3)
preview/z_0_1980.jpeg


<Figure size 640x480 with 1 Axes>

(183, 275, 3)


In [1]:
# MODEL
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(300, 300, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# the model so far outputs 3D feature maps (height, width, features)

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))
# COMPILE
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

Using TensorFlow backend.


In [6]:
batch_size = 16

train_datagen = ImageDataGenerator(
        width_shift_range=0.2,
        height_shift_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

test_datagen = ImageDataGenerator(
        width_shift_range=0.2,
        height_shift_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

# this is a generator that will read pictures found in subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        'train',  # this is the target directory
        target_size=(300, 300),  # all images will be resized to 300x300
        batch_size=batch_size,
        class_mode='binary')  # since we use binary_crossentropy loss, we need binary labels

# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        'validation',
        target_size=(300, 300),
        batch_size=batch_size,
        class_mode='binary')

Found 15 images belonging to 2 classes.
Found 4 images belonging to 2 classes.


In [7]:
# TRAINING
model.fit_generator(
        train_generator,
        steps_per_epoch=400 // batch_size,
        epochs=10,
        validation_data=validation_generator,
        validation_steps=160 // batch_size)

model.save_weights('10_epochs.h5')  # always save your weights after training or during training

Epoch 1/10
25/25 [==============================] - 34s 1s/step - loss: 9.2852 - acc: 0.4187 - val_loss: 7.9712 - val_acc: 0.5000
Epoch 2/10
25/25 [==============================] - 33s 1s/step - loss: 8.0252 - acc: 0.4987 - val_loss: 7.9712 - val_acc: 0.5000
Epoch 3/10
25/25 [==============================] - 33s 1s/step - loss: 8.6834 - acc: 0.4560 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 4/10
25/25 [==============================] - 33s 1s/step - loss: 8.2495 - acc: 0.4853 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 5/10
25/25 [==============================] - 33s 1s/step - loss: 8.2911 - acc: 0.4827 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 6/10
25/25 [==============================] - 36s 1s/step - loss: 8.0771 - acc: 0.4960 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 7/10
25/25 [==============================] - 32s 1s/step - loss: 7.6931 - acc: 0.5200 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 8/10
25/25 [==============================] - 32s 1s/step - loss: 7.9264 - acc: 0.50